In [1]:
import pandas as pd
import bs4
import re
import chardet


In [ ]:
with open("./diplomatic_directory.xml", 'rb') as f:
    print(chardet.detect(f.read()))

In [68]:
with open("./diplomatic_directory.xml") as file: # Use file to refer to the file object
    data = file.read()

In [70]:
data[:100]

'<?xml version="1.0" encoding="UTF-8"?>\n<TEI xmlns="http://www.tei-c.org/ns/1.0">\n    <teiHeader>\n   '

In [72]:
xmlDoc=bs4.BeautifulSoup(data, "xml")
for fullText in xmlDoc.children:
    print(type(fullText))

In [81]:
A_diplomats = fullText.get_text()[fullText.get_text().find("[A]"):fullText.get_text().find("[B]")].split("\n\n")[1:]

In [487]:
def split_dates_create_output_line(last_name, yob, yod, full_name, job, location, dates):
    if dates.find("-")!=-1:
        start, end = dates.split("-")[-2:]
        if not start.isdigit():
            start = ""
        if not end.isdigit():
            end = ""
        output = [last_name, yob, yod, full_name, job, location, start, end, ""]
    else:
        if "(did not proceed)" in dates:
            output = [last_name, yob, yod, full_name, job, location, dates[:4], dates[:4], "(did not proceed)" ]
            return output
        if not dates.isdigit():
            dates = ""
        output = [last_name, yob, yod, full_name, job, location, dates, dates, ""]
    return output

In [516]:
def parse_career(diplomatic_entry):
    if diplomatic_entry == [''] or len(diplomatic_entry)==1:
        return []
    return_list = []
    full_name = diplomatic_entry[0]#.split("")[0]
    last_name = full_name.split(', ')[0]
    
    getDob = re.compile("\((\d*)-(\d*)\)*")
    try:
        birthDeathObj = getDob.search(full_name)
        yob, yod = birthDeathObj.groups()
        full_name = full_name[:birthDeathObj.span()[0]].strip()
    except AttributeError:
        yob, yod = "", ""

    for posting in diplomatic_entry[1].split("; "):
        post = posting.split(', ')[:-1]#', '.join(posting.split(', ')[:-1])
        if post=="":
            continue
#         post = ', '.join(post)
        if len(post)==2:
            job = post[0]
            location = post[1]
        elif len(post)==1:
            
            if " to " in post[0]:
                post_split_on_to = post[0].split(" to ")
                if len(post_split_on_to)==2:
                    job, location = post[0].split(" to ")
                    job = job.strip()
                    location = location.strip()
                else:
                    job=post[0]
                    location=""
            else:
                job = ', '.join(post)
                location=""
        else:
            job = ', '.join(post)
            location=""
            
        # change the U.S.A to U.S.A etc
        if location.startswith("the ") or location.startswith("The "):
            location = location[4:]
            
        #edge cases    
        if job.find("Embassy, ")!=-1:
            location_of_start_Embassy = job.find("Embassy, ")
            location = job[location_of_start_Embassy+len("Embassy, "):]
            job = job[:location_of_start_Embassy+len("Embassy")]
            
        if job.find("Governor of ")!=-1:
            location_of_start_Governor = job.find("Governor of ")
            location = job[location_of_start_Governor+len("Governor of "):]
            job = job[:location_of_start_Governor+len("Governor")]
            
        if job.find("Consul-General, ")!=-1:
            location_of_start_ConsulGen = job.find("Consul-General, ")
            location = job.split(", ")[-1]
            job = "Consul-General"
            
        if job.find("Deputy High Commissioner, ")!=-1:
            location_of_start_HighCom = job.find("Deputy High Commissioner, ")
            location = job.split(", ")[-1]
            job = "Deputy High Commissioner"
            
        if job.find("High Commissioner in ")!=-1:
            location = job.split(" in ")[-1]
            job = "High Commissioner"
            
        if job.find("Counsellor, Legation, ")!=-1:
            location = job.split(", ")[-1]
            job = "Counsellor"
            
        if job.find("Counsellor, High Commission, ")!=-1:
            location = job.split(", ")[-1]
            job = "Counsellor"
            
        if job.find("Political Secretary, High Commission, ")!=-1:
            location = job.split(", ")[-1]
            job = "Political Secretary"
            
        if job.find("Official Secretary, High Commission, ")!=-1:
            location = job.split(", ")[-1]
            job =  "Official Secretary"
            
        if job.find("Governor-General of ")!=-1:
            location = job.split(" of ")[-1]
            job = "Governor-General"
            
        if job.find("Counsellor and Head of Chancery, High Commission, ")!=-1:
            location = job.split(" of ")[-1]
            job = "Counsellor and Head of Chancery"
            
        if job.find("Chancery, High Commission, ")!=-1:
            location = job.split(" of ")[-1]
            job = "Chancery"
            
        dates = posting.split(', ')[-1]
#         if ("and" in dates):
#             print("todo 'and' in dates")
        position_of_possible_and_in_dates = dates.find('and')
        if position_of_possible_and_in_dates == -1:
            output = split_dates_create_output_line(last_name, yob, yod, full_name, job, location, dates)
            return_list.append(output)
        else:
            first_posting_dates = dates[:position_of_possible_and_in_dates].strip()
            output = split_dates_create_output_line(last_name, yob, yod, full_name, job, location,first_posting_dates) 
            return_list.append(output)
            second_posting_dates = dates[position_of_possible_and_in_dates+3:].strip()
            output = split_dates_create_output_line(last_name, yob, yod, full_name, job, location,second_posting_dates)
            return_list.append(output)  
        
    return return_list

In [517]:
## Just the A's
# with open("tidy5.tsv", 'w') as f:
#     header = ["lastName", "yearOfBirth", "yearOfDeath", "fullName", "job", "location", "yearPostingStart", "yearPostingEnd", "notes"]
#     f.write("\t".join(header))
#     f.write("\n")
#     for tmp_dip in A_diplomats:
#         for entry in parse_career(tmp_dip.strip().split('\n')):
#             f.write("\t".join(entry))
#             f.write("\n")
#             print("FINAL", "\t".join(entry))

In [518]:
#problem with appostrophes
#A_diplomats[9]

In [519]:
A_diplomats[53]

'\nALLCHIN, SIR GEOFFREY C., K.B.E., C.M.G., M.C. (1895-1968)\nHead of Consular Department, Foreign Office, 1943-1947; Inspector-General, 1947-1949; Minister/Ambassador to Luxemburg, 1949-1955'

In [520]:
A_diplomats[107]

'ARTHUR, SIR GEOFFREY G., K.C.M.G. (1920-1984)\nCounsellor, Embassy, United Arab Republic, 1959-1963; Head of Permanent Under-Secretary’s Department, Foreign Office, 1963-1966; Head of Defence Department, Foreign Office, 1966; Ambassador to Kuwait, 1967-1968; Assistant Under-Secretary for Foreign Affairs (Middle East and North Africa), 1968-1970; Political Resident, Persian Gulf, 1970-1971; Deputy Under-Secretary for Foreign and Commonwealth Affairs (Defence and Intelligence), 1973-1975'

In [521]:
def getDiplomatsByInitial():
    listOfLetters = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","W","Y","Z", "SECTION B"]
    for i in range(len(listOfLetters)-1):
        start = listOfLetters[i]
        end = listOfLetters[i+1]
        if start == "Z":
            yield fullText.get_text()[fullText.get_text().find("["+start+"]"):fullText.get_text().find(end)].split("\n\n")[1:]
        else:
            yield fullText.get_text()[fullText.get_text().find("["+start+"]"):fullText.get_text().find("["+end+"]")].split("\n\n")[1:]

In [522]:
with open("diplomaticPosting.tsv", 'w') as f:
    header = ["lastName", "yearOfBirth", "yearOfDeath", "fullName", "job", "location", "yearPostingStart", "yearPostingEnd", "notes"]
    f.write("\t".join(header))
    f.write("\n")

    for dipRecs in getDiplomatsByInitial():
        for tmp_dip in dipRecs:
            for entry in parse_career(tmp_dip.strip().split('\n')):
                f.write("\t".join(entry))
                f.write("\n")
#                 print("FINAL", "\t".join(entry))